In [1]:
G_Debug = True   # if debug mode.  Debug = True. if Release mode, Debug = False.
G_Chrome_Hide = True
G_Debug_Pages = 2
G_Debug_Items = 5


G_CVS_FILE_NAME =  'tianjin.csv'

G_XLSX_FILE_NAME = 'tianjin2019.xlsx'

G_BID_ATTRIBUTE = ("id","status","start","end","title","consultPrice",
                                "marketPrice","currentPrice","bidCount","delayCount","applyCount","itemUrl")

G_CVS_HEADER =  ("name","start","end","bidNumber","title","originPrice",
                         "currentPrice","discount","unit","bidCount","delayCount",
                         "applyCount","area","areaA","areaB","itemUrl")


In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import json
import time
import csv
import pandas as pd
import re

In [3]:
# time
def getCurrentTimeStamp():
    t = time.time()
    print(time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(t)))
    return t


def printDuration(start, end):
    seconds = end - start
    m, s = divmod(seconds, 60)
    h, m = divmod(m, 60)
    print ("%02d:%02d:%02d" % (h, m, s))
    
    

# import time
# dt = "2019-06-03 10:00:00"
# #转换成时间数组
# timeArray = time.strptime(dt, "%Y-%m-%d %H:%M:%S")
# #转换成时间戳
# timestamp = time.mktime(timeArray)
# print (timestamp)

In [4]:
def log(page_id,item_id,status):
    print("page id: {0}, item id: {1}, status:{2}".format(page_id,item_id,status))

In [5]:
class DataManagement:
    def __init__(self):
        self.data = {}
        self.data["name"] = None
        
        global G_CVS_FILE_NAME
        global G_XLSX_FILE_NAME
        global G_CVS_HEADER
        
        self.cvs_file_name = G_CVS_FILE_NAME
        self.xlsx_file_name = G_XLSX_FILE_NAME
        self.cvs_head = G_CVS_HEADER
        
        self.__open_file()

    def __process_attri_name(self,attribute, data):
        index = data.find("通过")
        data = data[4:index]
        self.data[attribute] =  data     
            
    def __process_attri_area(self,attribute, data):
        left = data.find("建筑面积")
        if(left != -1):
            size = data[(left+4) : (left+22)]
            areas = re.findall(r"\d+\.\d*",size)
            self.data[attribute] = areas[0] if areas else 0
        else:
            self.data[attribute] = 0
        
    def __process_attri_date(self,attribute, data):
        time1 = int(int(data)/1000 )
        self.data[attribute] = str(time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time1)))
        
    def __process_attri_times(self,attribute, data):
        keyword_postion = data.find("卖")
        if(keyword_postion != -1):
            data = data[1:keyword_postion+1]
        else:
            data = "no content"
            
        self.data[attribute] = data



    def __process_attri_common(self,attribute, data):
        self.data[attribute] =  data        
     
    def __process_attri_url(self,attribute, data):
        url = "https:"+data
        self.data[attribute] =  url        

    
    def __clac_data(self):
        # 
        if (int(self.data["consultPrice"]) > 0):
            originPrice = self.data["consultPrice"]
        else:
            originPrice = self.data["marketPrice"]
        #
        if (int(originPrice)  == 0):
            self.data["discount"] = "error"
        else:
            self.data["discount"]  = float("%.2f" % float(int(self.data["currentPrice"])/originPrice))
 
       # change unit to from RMB yuan to  RMB Wan
        self.data["originPrice"]  = float("%.2f" % float(int(originPrice)/10000))        
        self.data["currentPrice"] = float("%.2f" % float(int(self.data["currentPrice"])/10000))

      # set area
        if(float(self.data["areaA"]) > 0):
            self.data["area"]  = self.data["areaA"]
            self.data["unit"] = float("%.2f" % float(int(self.data["currentPrice"])/float(self.data["area"]) ))
        elif(float(self.data["areaB"]) > 0):
            self.data["area"] = self.data["areaB"]
            self.data["unit"] = float("%.2f" % float(int(self.data["currentPrice"])/float(self.data["area"]) ))
        else:
            self.data["area"] = 0
            self.data["unit"] = 0
      ###    
        
    
    def set_data(self,attribute, data):
        if(attribute == "name"):
            self.__process_attri_name(attribute, data)
        elif(attribute == "areaA" or attribute == "areaB"):
            self.__process_attri_area(attribute, data)
        elif(attribute == "start" or attribute == "end"):
            self.__process_attri_date(attribute, data)
        elif(attribute == "bidNumber"):
            self.__process_attri_times(attribute, data)
        elif(attribute == "itemUrl"):
            self.__process_attri_url(attribute, data)
        else:
            self.__process_attri_common(attribute,data)    
    
    
    def get_data(self,attribute):
        return self.data[attribute]
    
    def __open_file(self):
        self.csv_file = open(self.cvs_file_name,"w+")
        self.writer = csv.writer(self.csv_file)        
        self.writer.writerow(self.cvs_head)

    def write_file(self):
        
        self.__clac_data()
        
        row = tuple(self.data[i] for i in self.cvs_head)
        
        self.writer.writerow(row)
    
    def close_file(self):
        self.csv_file.close()
        csv = pd.read_csv(self.cvs_file_name, encoding='utf-8')
        csv.to_excel(self.xlsx_file_name, sheet_name='data')
            


In [6]:
# time
startTime =  getCurrentTimeStamp()

2019-06-11 22:08:57


In [7]:
data = DataManagement()


In [8]:
#will bid
# sourceUrl = ("https://sf.taobao.com/item_list.htm?spm=a213w.7398504.filter.60.53db26cdCGMLqt&category=50025969"  
#                    "&auction_source=0&province=%CC%EC%BD%F2&sorder=1&st_param=-1&auction_start_seg=-1")
# #done bid
sourceUrl = ("https://sf.taobao.com/item_list.htm?category=50025969&auction_source=0&province=%CC%EC%BD%F2&sorder=2"
               "&st_param=-1&auction_start_seg=&auction_start_from=2019-01-01&auction_start_to=2019-07-10&&spm=a213w.3064813.9001.2")

In [9]:
#set headless
chrome_options=Options()
chrome_options.add_argument('--headless')
# Start the driver
if(G_Chrome_Hide):
    driver = webdriver.Chrome(options=chrome_options)
else:
    driver = webdriver.Chrome()
    

# Setup wait for later
wait = WebDriverWait(driver, 10)

driver.get(sourceUrl)

# Store the ID of the original window
original_window = driver.current_window_handle

# Check we don't have other windows open already
assert len(driver.window_handles) == 1

wait.until(EC.presence_of_element_located((By.CLASS_NAME, "page-total")))

<selenium.webdriver.remote.webelement.WebElement (session="21c997f6a5f22bddc7686fcbf2816f8c", element="0.8757642964377286-1")>

In [10]:
# TEST

if(G_Debug):
    pages = G_Debug_Pages
else:
    pages = driver.find_element_by_class_name("page-total").text


In [11]:
for page in range(1,int(pages)+1):
    if(page > 1):
        driver.find_element_by_link_text(str(page)).click()
        time.sleep(1)
    
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, "sf-pai-item-list")))
    jsonData =  json.loads(driver.find_element_by_id("sf-item-list-data").get_attribute('innerHTML'))
    page_data = jsonData["data"]
    
    if(G_Debug):
        items = G_Debug_Items
    else:
        items = len(page_data)


    for item in range(items):
        log(page,item,page_data[item]["status"])
        for key,value in page_data[item].items():
            if key in G_BID_ATTRIBUTE:
                data.set_data(key,value)

        # Click the link which opens in a new window
        driver.find_element_by_css_selector("#pai-item-"+str(data.get_data("id"))).click()
        time.sleep(1) # have to wait sometime
        # Wait for the new window or tab
        wait.until(EC.number_of_windows_to_be(2))
        handles = driver.window_handles
        driver.switch_to.window(handles[1])
        item_url = driver.current_url

       # bid times
        wait.until(EC.presence_of_element_located((By.XPATH, "/html[1]/body[1]/div[3]/div[4]/div[1]/div[1]/h1[1]")))
        data.set_data("bidNumber",driver.find_element_by_xpath("/html[1]/body[1]/div[3]/div[4]/div[1]/div[1]/h1[1]").get_attribute('textContent'))
        #TODO: degfine new field

       # get area
        driver.get(item_url+"#NoticeDetail")
        time.sleep(1)
        data.set_data("areaA",driver.find_element_by_id("J_NoticeDetail").get_attribute('textContent'))
        
       ###  Tip
        driver.get(item_url+"#RemindTip")
        time.sleep(1)
        data.set_data("areaB",driver.find_element_by_id("J_desc").get_attribute('textContent'))
              
        if(data.get_data("status") == "done"):
            driver.get(item_url+"#J_Confirmation")
            wait.until(EC.presence_of_element_located((By.CLASS_NAME, "content-wrap")))
            name = driver.find_element_by_class_name("c-content").text
        else:
            name = "用户姓名失败通过"
        data.set_data("name",name)
                 
        data.write_file()
        driver.close()
        driver.switch_to.window(handles[0])

page id: 1, item id: 0, status:done
page id: 1, item id: 1, status:done
page id: 1, item id: 2, status:failure
page id: 1, item id: 3, status:failure
page id: 1, item id: 4, status:failure
page id: 2, item id: 0, status:done
page id: 2, item id: 1, status:done
page id: 2, item id: 2, status:done
page id: 2, item id: 3, status:done
page id: 2, item id: 4, status:done


In [12]:
driver.quit()

In [13]:
data.close_file()

In [14]:
endTime = getCurrentTimeStamp()

printDuration(startTime, endTime)

2019-06-11 22:09:43
00:00:46
